# Método dos Elementos Finitos - Trabalho 4

Universidade Federal Fluminense

Disciplina ministrada pelo Prof. Marco Ferro

<marcoferro@id.uff.br>

Aluno Noé de Lima

<noe_lima@id.uff.br>

Este trabalho visa aplicar o MEF ao Estado Plano de Tensões (EPT).

Primeiro semestre de 2020

\vfill

A célula a seguir configura o Jupyter-Notebook para exibir as equações matemáticas no formato do ambiente $\LaTeX$ e importa as bibliotecas necessárias.

In [1]:
import numpy as np
import sympy as sp
import pandas as pd
from sympy.vector import CoordSys3D, divergence
sp.init_printing(use_latex='mathjax',latex_mode='equation*')
!uname -a

Linux DESKTOP-CR7O8A2 4.19.104-microsoft-standard #1 SMP Wed Feb 19 06:37:35 UTC 2020 x86_64 x86_64 x86_64 GNU/Linux


\cleardoublepage

\tableofcontents

\cleardoublepage

# Introdução

O Estado Plano de Tenção - EPT ou o Estado Plano de Deformação - EPD, caracterizam-se por serem estruturas planares (no plano $xy$), com carregaemntos, reações de apoio, restrições e deformações igualmente planares. Tais estruturas têm, na direção $z$, espessura fixa constante $t$. As condições que definem e diferenciam o EPT e o EPD estão relacionadas às restrições na direção $z$.

Temos, na dereção $z$, das faces do plano, as seguintes variáveis de Tensão $\times$ Deformação: $\sigma_{z}$ e $\epsilon_{z}$.

Portanto, segue que:

No EPT: $\sigma_{z}=0$;

No EPD: $\epsilon_{z}=0$.

O EPT tem, portanto, suas faces livres, sem tensões de reação e liberdade de deformação. Tal é a situação em chapas, onde uma estrutura plana recebe carregamentos no mesmo plano da face, e não recebe carregamentos transversais (como nas placas ou lages). Nessa configuração, a espessura $t$ da chapa pode variar, pois não há tensão de reação na direção $z$.

Já o EPD é caracterizado por uma restrição de deformação, já que há reação de tensão na direção $z$ que restringe a deformação. Nessa configuração, portanto, a espessura $t$ não varia. Este é o caso onde o plano compõe uma seção transversão de uma estrutura maior, como um muro de arrimo ou uma barragem de represa. Neste caso, a espessura é fixa e definida como $t=1$, sendo as tensões definidas em termos de comprimento de muro ou barragem.

As variáveis simbólicas definidas abaixo serão utilizadas na análise subsequente.

In [2]:
E,nu = sp.symbols('E,nu')
G = E/(2*(1+nu))

Sendo,

* $E$ o Módulo de Elasticidade Longitudinal;

* $\nu$ a constante de Poisson do material; elastica; e

* $G$ o Módulo de Elasticidade Transversal.

In [3]:

sigma_x,sigma_y,sigma_z = sp.symbols('sigma_x,sigma_y,sigma_z')
tau_xy,tau_yz,tau_zx = sp.symbols('tau_xy,tau_yz,tau_zx')
epsilon_x,epsilon_y,epsilon_z = sp.symbols('epsilon_x,epsilon_y,epsilon_z')
gamma_xy,gamma_yz,gamma_zx = sp.symbols('gamma_xy,gamma_yz,gamma_zx')

Onde,

* $\sigma_{x}$, $\sigma_{y}$ e $\sigma_{z}$ são, respectivamente, as tensões normais nas direções $x$, $y$ e $z$;

* $\tau_{xy}$, $\tau_{yz}$ e $\tau_{zx}$ são, respectivamente, as tensões cisalhantes nas direções $xy$, $yz$ e $zx$;

* $\varepsilon_{x}$, $\varepsilon_{y}$ e $\varepsilon_{z}$ são, respectivamente, as deformações lineares em $x$ e $y$ e $z$;

* $\gamma_{xy}$, $\gamma_{yz}$ e $\gamma_{zx}$ são, respectivamente, as deformações angulares nas direções $xy$, $yz$ e $zx$.

A partir das restrições do sistema bidimensional, temos o seguinte Vetor de Tensões $\vec{\sigma}$:

In [4]:
sigma = sp.Matrix([sigma_x,sigma_y,tau_xy])
display(sigma)

⎡ σₓ ⎤
⎢    ⎥
⎢σ_y ⎥
⎢    ⎥
⎣τ_xy⎦

Bem como o Vetor de Deformações $\vec{\varepsilon}$:

In [5]:
epsilon = sp.Matrix([epsilon_x,epsilon_y,gamma_xy])
display(epsilon)

⎡ εₓ ⎤
⎢    ⎥
⎢ε_y ⎥
⎢    ⎥
⎣γ_xy⎦

Temos também a matriz $\mathbf{D}$, em função de $E_{var}$ e $\nu_{var}$:

In [6]:
E_var,nu_var = sp.symbols('E_var,nu_var')
D_var = (E_var/(1-nu_var**2))*sp.Matrix([[1,nu_var,0],[nu_var,1,0],[0,0,(1-nu_var)/2]])
display(D_var)

⎡   Eᵥₐᵣ    Eᵥₐᵣ⋅νᵥₐᵣ                 ⎤
⎢─────────  ─────────         0       ⎥
⎢        2          2                 ⎥
⎢1 - νᵥₐᵣ   1 - νᵥₐᵣ                  ⎥
⎢                                     ⎥
⎢Eᵥₐᵣ⋅νᵥₐᵣ     Eᵥₐᵣ                   ⎥
⎢─────────  ─────────         0       ⎥
⎢        2          2                 ⎥
⎢1 - νᵥₐᵣ   1 - νᵥₐᵣ                  ⎥
⎢                                     ⎥
⎢                           ⎛1   νᵥₐᵣ⎞⎥
⎢                      Eᵥₐᵣ⋅⎜─ - ────⎟⎥
⎢                           ⎝2    2  ⎠⎥
⎢    0          0      ───────────────⎥
⎢                                 2   ⎥
⎣                         1 - νᵥₐᵣ    ⎦

A equação do sistema, então, fica:

In [7]:
eq = sp.Eq(sigma, D_var*epsilon)
display(eq)

         ⎡ Eᵥₐᵣ⋅εₓ    Eᵥₐᵣ⋅ε_y⋅νᵥₐᵣ⎤
         ⎢───────── + ─────────────⎥
         ⎢        2             2  ⎥
         ⎢1 - νᵥₐᵣ      1 - νᵥₐᵣ   ⎥
         ⎢                         ⎥
         ⎢Eᵥₐᵣ⋅εₓ⋅νᵥₐᵣ    Eᵥₐᵣ⋅ε_y ⎥
⎡ σₓ ⎤   ⎢──────────── + ───────── ⎥
⎢    ⎥   ⎢         2             2 ⎥
⎢σ_y ⎥ = ⎢ 1 - νᵥₐᵣ      1 - νᵥₐᵣ  ⎥
⎢    ⎥   ⎢                         ⎥
⎣τ_xy⎦   ⎢            ⎛1   νᵥₐᵣ⎞   ⎥
         ⎢  Eᵥₐᵣ⋅γ_xy⋅⎜─ - ────⎟   ⎥
         ⎢            ⎝2    2  ⎠   ⎥
         ⎢  ────────────────────   ⎥
         ⎢               2         ⎥
         ⎣       1 - νᵥₐᵣ          ⎦

# Estado Plano de Tensão - EPT

As condições de contorno do EPT, conforme definido na introdução, podem ser definidas matematicamente como:

\begin{equation*}
    \left\{\begin{matrix}
        \sigma_{z}  & = &   0 \\
        \tau_{yz}   & = &   0 \\
        \tau_{zx}   & = &   0
    \end{matrix}\right.
\end{equation*}

E,

\begin{equation*}
    \varepsilon_{z} = -\frac{\nu}{E}\left(\sigma_{x}+\sigma_{y}\right) \neq 0
\end{equation*}

Nessas condições, temos:

\begin{equation*}
    \left\{\begin{matrix}
        E_{var}     & = &   E \\
        \nu_{var}   & = &   \nu
    \end{matrix}\right.
\end{equation*}

Logo, $\mathbf{D}_{EPT}$ =

In [8]:
ept = eq.subs(E_var,E).subs(nu_var,nu)
display(ept)

         ⎡ E⋅εₓ    E⋅ε_y⋅ν⎤
         ⎢────── + ───────⎥
         ⎢     2         2⎥
         ⎢1 - ν     1 - ν ⎥
         ⎢                ⎥
         ⎢E⋅εₓ⋅ν   E⋅ε_y  ⎥
⎡ σₓ ⎤   ⎢────── + ────── ⎥
⎢    ⎥   ⎢     2        2 ⎥
⎢σ_y ⎥ = ⎢1 - ν    1 - ν  ⎥
⎢    ⎥   ⎢                ⎥
⎣τ_xy⎦   ⎢        ⎛1   ν⎞ ⎥
         ⎢ E⋅γ_xy⋅⎜─ - ─⎟ ⎥
         ⎢        ⎝2   2⎠ ⎥
         ⎢ ────────────── ⎥
         ⎢          2     ⎥
         ⎣     1 - ν      ⎦

# Estado Plano de Deformação

As condições de contorno do EPD, conforme definido na introdução, podem ser definidas matematicamente como:

\begin{equation*}
    \left\{\begin{matrix}
        \varepsilon_{z}    & = &   0 \\
        \gamma_{yz}     & = &   0 \\
        \gamma_{zx}     & = &   0
    \end{matrix}\right.
\end{equation*}

E,

\begin{equation*}
    \sigma_{z} = \nu\left(\sigma_{x}+\sigma_{y}\right) \neq 0
\end{equation*}

Nessas condições, temos:

\begin{equation*}
    \left\{\begin{matrix}
        E_{var}     & = &   \frac{E}{1-\nu^{2}} \\
        \nu_{var}   & = &   \frac{\nu}{1-\nu}
    \end{matrix}\right.
\end{equation*}

Logo, $\mathbf{D}_{EPD}$ =

In [9]:
epd = eq.subs(E_var,E/(1-nu**2)).subs(nu_var,nu/(1-nu))
display(epd)

         ⎡           E⋅εₓ                          E⋅ε_y⋅ν             ⎤
         ⎢───────────────────────── + ─────────────────────────────────⎥
         ⎢         ⎛      2       ⎞                    ⎛      2       ⎞⎥
         ⎢⎛     2⎞ ⎜     ν        ⎟           ⎛     2⎞ ⎜     ν        ⎟⎥
         ⎢⎝1 - ν ⎠⋅⎜- ──────── + 1⎟   (1 - ν)⋅⎝1 - ν ⎠⋅⎜- ──────── + 1⎟⎥
         ⎢         ⎜         2    ⎟                    ⎜         2    ⎟⎥
         ⎢         ⎝  (1 - ν)     ⎠                    ⎝  (1 - ν)     ⎠⎥
         ⎢                                                             ⎥
         ⎢              E⋅εₓ⋅ν                          E⋅ε_y          ⎥
         ⎢───────────────────────────────── + ─────────────────────────⎥
⎡ σₓ ⎤   ⎢                 ⎛      2       ⎞            ⎛      2       ⎞⎥
⎢    ⎥   ⎢        ⎛     2⎞ ⎜     ν        ⎟   ⎛     2⎞ ⎜     ν        ⎟⎥
⎢σ_y ⎥ = ⎢(1 - ν)⋅⎝1 - ν ⎠⋅⎜- ──────── + 1⎟   ⎝1 - ν ⎠⋅⎜- ──────── + 1⎟⎥
⎢    ⎥   ⎢                 ⎜         2    ⎟        

# Cálculo das Tensões

Tanto para EPT quanto para EPD, temos a seguinte equação para o cálculo das tensões (Derivada da Lei de Hooke):

\begin{equation*}
    \vec{\sigma} = \mathbf{D}\vec{\varepsilon}
\end{equation*}

Bem como,

\begin{equation*}
    \vec{\varepsilon} = \mathbf{L}\vec{u}
\end{equation*}

Onde,

* $\mathbf{L}\rightarrow$ é a matriz de operadores que relaciona os deslocamentos com as deformaçoes.

Como $$\bar{u} = \sum_{i=1}^{N}{N_{i}u_{i}}$$ tem-se:

\begin{equation*}
    \mathbf{L} =
    \left[\begin{matrix}
        \frac{\partial}{\partial x} & 0 \\
        0 & \frac{\partial}{\partial y} \\
        \frac{\partial}{\partial y} & \frac{\partial}{\partial x}
    \end{matrix}\right]
\end{equation*}

\begin{equation*}
    \vec{u} =
    \left\{\begin{matrix}
        u \\
        v
    \end{matrix}\right\}
\end{equation*}

\begin{equation*}
    \vec{\varepsilon} =
    \left\{\begin{matrix}
        \varepsilon_{x} \\
        \varepsilon_{y} \\
        \gamma_{xy}
    \end{matrix}\right\}
\end{equation*}

# Método dos Elementos Finitos

O Elemento bidimensional a ser estudado é o <b>Triangular de Deformação Constante</b>, ou <b><i>Constant Strain Triangle - CST</i></b>, que possui como funções de interpolação $N_{i}$, para os deslocamentos $u$ e $v$, polinômios do primeiro grau em $x$ e $y$.

Portanto,

\begin{equation*}
    \left\{\begin{matrix}
        u \\ v
    \end{matrix}\right\} =
    \left[\begin{matrix}
        1 & x & y & 0 & 0 & 0 \\
        0 & 0 & 1 & 1 & x & y
    \end{matrix}\right]
    \left\{\begin{matrix}
        a_{1} \\ a_{2} \\ a_{3} \\ b_{1} \\ b_{2} \\ b_{3}
    \end{matrix}\right\}
\end{equation*}

As funções $N_{i}$ têm a seguinte forma:

\begin{equation*}
    N_{1}\left(x,y\right) = \frac{1}{2A}
    \left[ \left(x_{2}y_{3} - x_{3}y_{2}\right) +
    \left(y_{2}-y_{3}\right)x +
    \left(x_{3}-x_{2}\right)y\right]
\end{equation*}

\begin{equation*}
    N_{2}\left(x,y\right) = \frac{1}{2A}
    \left[ \left(y_{1}x_{3} - y_{3}x_{1}\right) +
    \left(y_{3}-y_{1}\right)x +
    \left(x_{1}-x_{3}\right)y\right]
\end{equation*}

\begin{equation*}
    N_{3}\left(x,y\right) = \frac{1}{2A}
    \left[ \left(x_{1}y_{2} - x_{2}y_{1}\right) +
    \left(y_{1}-y_{2}\right)x +
    \left(x_{2}-x_{1}\right)y\right]
\end{equation*}

Sendo $A$ a área do triiângulo, dada por:

\begin{equation*}
    A = \frac{1}{2}\times \det{
    \left[\begin{matrix}
        1 & 1 & 1 \\
        x_{1} & x_{2} & x_{3} \\
        y_{1} & y_{2} & y_{3}
    \end{matrix}\right]}
\end{equation*}

As equações acima para $N_{i}$ podem ser resumidas em:

\begin{equation*}
    N_{i}\left(x,y\right) = \frac{1}{2A}
    \left[ \left(x_{j}y_{k} - x_{k}y_{j}\right) +
    \left(y_{j}-y_{k}\right)x +
    \left(x_{k}-x_{j}\right)y\right]
\end{equation*}

Sendo,

\begin{equation*}
    \left\{\begin{matrix}
        i & = & 1, & 2, & 3; \\
        j & = & 2, & 3, & 1; \\
        k & = & 3, & 1, & 2;
    \end{matrix}\right.
\end{equation*}

A matriz $\mathbf{N}$ é, então, formada:

\begin{equation*}
    \mathbf{N} =
    \left[\begin{matrix}
        N_{1} & 0 & N_{2} & 0 & N_{3} & 0 \\
        0 & N_{1} & 0 & N_{2} & 0 & N_{3}
    \end{matrix}\right]
\end{equation*}

Mas $\vec{\varepsilon} = \mathbf{L}\vec{u}$. Como $\vec{u} = \mathbf{N}\vec{u}_{i}$, tem-se $$\vec{\varepsilon} = \mathbf{L}\mathbf{N}\vec{u}_{i}$$

Fazendo-se $\mathbf{B} = \mathbf{L}\mathbf{N}$, tem-se:

\begin{equation*}
    \mathbf{B} =
    \left[\begin{matrix}
        \frac{\partial}{\partial x} & 0 \\
        0 & \frac{\partial}{\partial y} \\
        \frac{\partial}{\partial y} & \frac{\partial}{\partial x}
    \end{matrix}\right]
    \left[\begin{matrix}
        N_{1} & 0 & N_{2} & 0 & N_{3} & 0 \\
        0 & N_{1} & 0 & N_{2} & 0 & N_{3}
    \end{matrix}\right]
\end{equation*}

\begin{equation*}
    \mathbf{B} =
    \left[\begin{matrix}
        \frac{\partial N_{1}}{\partial x} & 0 & \frac{\partial N_{2}}{\partial x} & 0 & \frac{\partial N_{3}}{\partial x} & 0 \\
        0 & \frac{\partial N_{1}}{\partial y} & 0 & \frac{\partial N_{2}}{\partial y} & 0 & \frac{\partial N_{3}}{\partial y} \\
        \frac{\partial N_{1}}{\partial y} & \frac{\partial N_{1}}{\partial x} & \frac{\partial N_{2}}{\partial y} & \frac{\partial N_{2}}{\partial x} & \frac{\partial N_{3}}{\partial y} & \frac{\partial N_{3}}{\partial x}
    \end{matrix}\right]
\end{equation*}

Que resulta em:

\begin{equation*}
    \mathbf{B} =
    \left[\begin{matrix}
        \left(y_{2}-y_{3}\right) & 0 & \left(y_{3}-y_{1}\right) & 0 & \left(y_{1}-y_{2}\right) & 0 \\
        0 & \left(x_{3}-x_{2}\right) & 0 & \left(x_{1}-x_{3}\right) & 0 & \left(x_{2}-x_{1}\right) \\
        \left(x_{3}-x_{2}\right) & \left(y_{2}-y_{3}\right) & \left(x_{1}-x_{3}\right) & \left(y_{3}-y_{1}\right) & \left(x_{2}-x_{1}\right) & \left(y_{1}-y_{2}\right)
    \end{matrix}\right]
\end{equation*}

Tem-se que:

\begin{equation*}
    \mathbf{K}_{e} = t\mathbf{B}^{T}\mathbf{D}\mathbf{B}\times A
\end{equation*}

No <b>EPT</b>, $t$ é a espessura do elemento (chapa); no <b>EPD</b>, $t=1$.

# Exemplo 1

Calcular como EPD e EPT uma estrutura de área retangular de largura $75~mm$ em $x$ e altura $50~mm$ em $y$. Considerar apoio de segundo gênero nos pontos $(0,~0)$ e $(0,~50)$ e uma carga pontual vertical de $4450~N$ para baixo no ponto $(75,~50)$.

Dados:

\begin{equation*}
    \left\{\begin{matrix}
        t & = & 13~mm \\
        E & = & 207~GPa \\
        \nu & = & 0,25
    \end{matrix}\right.
\end{equation*}

A malha considerada será a seguinte: